# Explore time dependent deformations

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os,sys
from clawpack.geoclaw import topotools,kmltools
from scipy.interpolate import LinearNDInterpolator
import obspy
from clawpack.visclaw import animation_tools
from IPython.display import HTML


In [ ]:
#CHT = os.environ['CHT'] # path to CopesHubTsunamis directory
CHT = '/Users/rjl/git/CopesHubTsunamis' # or hard-wired
sys.path.insert(0,CHT+'/common_code')  # add to search path
import dtopotools # local version

## Select an event:

In [ ]:
#name = 'buried-locking-skl16-shallow'
#name = 'buried-random-str10-shallow'
name = 'buried-locking-str10-deep'
print('Rupture name: ',name)

## Read static displacement

In [ ]:
datadir = 'vertical_displacements'
fname_orig = 'vert_displacements_all_xgrid_' + name
path_orig = os.path.join(datadir, fname_orig)
lon,lat,zdisp = loadtxt(path_orig, skiprows=1,usecols=[1,2,3],unpack=True)

In [ ]:
zdisp.shape

In [ ]:
ii = argmax(zdisp)
print('max zdisp[%i] = %.1f m at x = %.4f, y = %.4f' \
      % (ii,zdisp[ii],lon[ii],lat[ii]))

## Read waveforms

In [ ]:
waveforms = obspy.read('time_dependent_zdisp/XGRID_%s_Z.h5' % name)

In [ ]:
len(waveforms)

In [ ]:
waveforms = waveforms.sort(['station'])

In [ ]:
tr = waveforms[ii]
t = tr.times()
id = tr.id
dz = tr.data
print('max dz from waveforms[%i] is %.1f m' % (ii,dz.max()))
print('final dz from waveforms[%i] is %.1f m' % (ii,dz[-1]))

In [ ]:
plot(t, dz, 'b')
grid(True)
title('waveforms[%i]' % ii)
xlabel('seconds')
ylabel('meters')

## Create time-dependent dtopo object:

In [ ]:
mx = 6*120 + 1  # 30 arcsec
my = 10*120 + 1  # 30 arcsec
x = linspace(-128.5,-122.5,mx)
y  = linspace(40,50,my)
X,Y = meshgrid(x,y)

dtopo = dtopotools.DTopography()
dtopo.X = X
dtopo.Y = Y
dtopo.times = tr.times()
ntimes = len(dtopo.times)
dZshape = (ntimes,X.shape[0],X.shape[1])
dtopo.dZ = empty(dZshape)

points = vstack((lon,lat)).T

for k in range(ntimes):
    # interpolate to uniform grid at each time:
    dispk = [w.data[k] for w in waveforms]
    dz_fcn = LinearNDInterpolator(points, dispk, fill_value=0.)
    dtopo.dZ[k,:,:] = dz_fcn(X,Y)

In [ ]:
dtopo.dZ.shape

### Save dtopo file for GeoClaw:

In [ ]:
fname_dtopo = name + '.dtt3'
dtopo.write(fname_dtopo, 3)

In [ ]:
dtopo.times

### Save dtopo file for instantaneous displacement

In [ ]:
dtopoS = dtopotools.DTopography()
dtopoS.X = X
dtopoS.Y = Y
dtopoS.times = [0.]
dZshape = (1,X.shape[0],X.shape[1])
dtopoS.dZ = empty(dZshape)
dtopoS.dZ[0,:,:] = dtopo.dZ[-1,:,:]

fname_dtopo = name + '_instant.dtt3'
dtopoS.write(fname_dtopo, 3)
print('Created ',fname_dtopo)

## Make plots and animations:

In [ ]:
coast = load('/Users/rjl/git/clawpack/geoclaw/scratch/pacific_shorelines_east_4min.npy')
x_coast = coast[:,0] - 360.
y_coast = coast[:,1]

In [ ]:
dZ_max = abs(dtopo.dZ).max()
cmax_dZ = round(dZ_max)
print('maximum |dZ| = %.2f m, setting cmax_dZ = %.1f' % (dZ_max,cmax_dZ))

In [ ]:
# plot at one time to test:
fig,ax = subplots(figsize=(6,7))
plot(x_coast,y_coast,'g')
axis([-128.5,-122,40,50])
t = 120
dtopo.plot_dZ_colors(t,axes=ax,cmax_dZ=cmax_dZ,dZ_interval=100)
title('%i seconds\n%s' % (t,name))
grid(True);

In [ ]:
figs = []
for k,t in enumerate(dtopo.times):
    fig,ax = plt.subplots(figsize=(6,7))
    plot(x_coast,y_coast,'g')
    axis([-128.5,-122,40,50])
    dtopo.plot_dZ_colors(t,axes=ax,cmax_dZ=cmax_dZ,dZ_interval=100)
    title('%i seconds\n%s' % (t,name))
    grid(True)
    figs.append(fig)
    close(fig)

In [ ]:
anim = animation_tools.animate_figs(figs, figsize=(7,8))

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
# make mp4 file:
fname = name + '.mp4'
animation_tools.make_mp4(anim, file_name=fname)

## plot transect of dz at a sequence of times:

In [ ]:
y0 = 47.5
j = where(y<y0)[0].max()
for k in range(6,15):
    plot(x,dtopo.dZ[k,j,:],label='%.0fs' % dtopo.times[k])
legend(loc='upper left')
title('Transect at y = %.1f\n%s' % (y0,name))
grid(True)
fname = name + '_y%s.png' % str(y0).replace('.','-')
savefig(fname)
print('Created ',fname)

In [ ]:
dtopo.dZ.max()